## Data Ingestion

- Currently using only Word and WordNet Extension columns
- Aggregating & grouping details into json for easy use

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv('dict4dataset_04-23-22.csv')

In [3]:
df

,Strategy Name,Strategy No.,Category No.,Category Name,Semantic Anchor,Word,WordNet Extension,Phrase Extension,Reprogramming
0,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depress,/,NaN,L2d: No Reprogramming
1,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depressed,"gloomy, grim, blue, dispirited, down, downcast...",NaN,NaN
2,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depression,/,NaN,NaN
3,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Dyshphoria,NaN,NaN,NaN,NaN
4,Understanding Common Symptom Indicators,L2d,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
82,Understanding Healthy Patterns,L2f,2.0,Coherent narrative words,NaN,therefore,NaN,NaN,NaN
83,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,help,NaN,NaN,NaN
84,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,improve,NaN,NaN,NaN
85,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,work on,NaN,NaN,NaN


In [4]:
df_clean = df.replace({"Word": "/", "WordNet Extension": "/"}, np.nan) \
             .dropna(subset=['Category No.']) \
             .astype({'Category No.': 'int32'}) \
             .sort_values(["Strategy No."])

df_clean

,Strategy Name,Strategy No.,Category No.,Category Name,Semantic Anchor,Word,WordNet Extension,Phrase Extension,Reprogramming
34,Understanding Judgement,L2a,1,Positive Adjectives,NaN,brave,NaN,xx is adj,...because
36,Understanding Judgement,L2a,2,Negative Adjectives,NaN,egoistic,NaN,NaN,NaN
39,Understanding Cognitive Distortion,L2b,1,Should Statement,should,should,NaN,NaN,-> can/choose/want to/prefer/would like to/pla...
41,Understanding Cognitive Distortion,L2b,2,All or Nothing Thinking / Overgeneralization,absolutive words,never,NaN,NaN,NaN
42,Understanding Cognitive Distortion,L2b,2,All or Nothing Thinking / Overgeneralization,absolutive words,always,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
81,Understanding Healthy Patterns,L2f,2,Coherent narrative words,NaN,because,NaN,NaN,NaN
82,Understanding Healthy Patterns,L2f,2,Coherent narrative words,NaN,therefore,NaN,NaN,NaN
83,Understanding Healthy Patterns,L2f,3,Constructive Self-Talk,NaN,help,NaN,NaN,NaN
84,Understanding Healthy Patterns,L2f,3,Constructive Self-Talk,NaN,improve,NaN,NaN,NaN


In [5]:
df_clean.columns

Index(['Strategy Name', 'Strategy No.', 'Category No.', 'Category Name',
       'Semantic Anchor', 'Word', 'WordNet Extension', 'Phrase Extension',
       'Reprogramming'],
      dtype='object')

In [9]:
groupped = df_clean.groupby(["Strategy No.", "Category No.", "Category Name", "Semantic Anchor"], dropna=False) \
                   .apply(lambda x: x[["Word","WordNet Extension","Phrase Extension", "Reprogramming"]].to_json(orient='columns'))

In [10]:
isinstance(groupped, pd.Series)

True

In [19]:
# groupped.groups
# groupped.index
for index, value in groupped.items():
    print(f"Index : {index}, Value : {json.loads(value)}")

Index : ('L2a', 1, 'Positive Adjectives', nan), Value : {'Word': {'34': 'brave'}, 'WordNet Extension': {'34': None}, 'Phrase Extension': {'34': 'xx is adj'}, 'Reprogramming': {'34': '...because'}}
Index : ('L2a', 2, 'Negative Adjectives', nan), Value : {'Word': {'36': 'egoistic'}, 'WordNet Extension': {'36': None}, 'Phrase Extension': {'36': None}, 'Reprogramming': {'36': None}}
Index : ('L2b', 1, 'Should Statement', 'should'), Value : {'Word': {'39': 'should'}, 'WordNet Extension': {'39': None}, 'Phrase Extension': {'39': None}, 'Reprogramming': {'39': '-> can/choose/want to/prefer/would like to/plan to'}}
Index : ('L2b', 2, 'All or Nothing Thinking / Overgeneralization', 'absolutive words'), Value : {'Word': {'41': 'never', '42': 'always', '43': 'forever', '45': 'all', '46': 'everyone', '47': 'everything', '48': 'anyone', '49': 'anything', '50': 'only', '51': 'totally', '52': 'no one', '44': 'again', '54': 'nothing', '53': 'none'}, 'WordNet Extension': {'41': None, '42': None, '43': 

In [103]:
    # TODO: Define target structure:
    
#     dict = [
#       'L2b': {
#          'Category Number' : 1,
#          'Category Name' : 'Should Statement',
#          'Anchors': {
#             'Depressed Mood': {
#                'Words' : ["should"],
#                'WordNet Extension' : [],
#                'Phrase Extension': [],
#                'Reprogramming' : []    
#             },
#             'Dysphoria': {
#                'Words' : [],
#                'WordNet Extension' : [],
#                'Phrase Extension': [],
#                'Reprogramming' : []    
#             }
#          },
#       }, 
    
#       {  'L2c' : {
#          'Cat Number' : 2,
#          'Cat Name' : 'Should Statement',
#          'Words' : ["should"],
#          'WordNet Extension' : [],
#          'Phrase Extension': [],
#          'Reprogramming' : []
#       }
#     ]

In [21]:
agg = []
# for index, value in groupped.items():
#     out = { 
#         "Strategy No." : index[0],
#         "Category No." : index[1],
#         "Semantic Anchor" : index[2],
#         "Words" : json.loads(value)["Word"],
#         "WordNet Extension" : json.loads(value)["WordNet Extension"],
#     }
#     agg.append(out)
# print(agg)

for index, value in groupped.items():
    out = { 
        "strategy_code" : index[0],
        "category_number" : index[1],
        "semantic_anchor" : index[2],
        "words" : [i for i in list(json.loads(value)["Word"].values()) if i],
        "wordnet_ext" : [i for i in list(json.loads(value)["WordNet Extension"].values()) if i],
        "phrase_ext" : [ i for i in list(json.loads(value)["Phrase Extension"].values()) if i],
        "reprogramming" : [ i for i in list(json.loads(value)["Reprogramming"].values()) if i],
    }
    agg.append(out)
print(agg)


[{'strategy_code': 'L2a', 'category_number': 1, 'semantic_anchor': 'Positive Adjectives', 'words': ['brave'], 'wordnet_ext': [], 'phrase_ext': ['xx is adj'], 'reprogramming': ['...because']}, {'strategy_code': 'L2a', 'category_number': 2, 'semantic_anchor': 'Negative Adjectives', 'words': ['egoistic'], 'wordnet_ext': [], 'phrase_ext': [], 'reprogramming': []}, {'strategy_code': 'L2b', 'category_number': 1, 'semantic_anchor': 'Should Statement', 'words': ['should'], 'wordnet_ext': [], 'phrase_ext': [], 'reprogramming': ['-> can/choose/want to/prefer/would like to/plan to']}, {'strategy_code': 'L2b', 'category_number': 2, 'semantic_anchor': 'All or Nothing Thinking / Overgeneralization', 'words': ['never', 'always', 'forever', 'all', 'everyone', 'everything', 'anyone', 'anything', 'only', 'totally', 'no one', 'again', 'nothing', 'none'], 'wordnet_ext': ['everybody', 'anybody'], 'phrase_ext': [], 'reprogramming': []}, {'strategy_code': 'L2b', 'category_number': 3, 'semantic_anchor': 'Cata

## FOR SAVING:

In [22]:
import datetime
ts = datetime.datetime.now().isoformat()

In [23]:
with open(ts+'dict.json', 'w', encoding='utf-8') as f:
    json.dump(agg, f, ensure_ascii=False, indent=4)